# Scaling content review operations with multi-agent workflow

## Overview

This tutorial demonstrates how to deploy a multi-agent content analysis system using Amazon Bedrock AgentCore Runtime. The system analyzes technical blog content for freshness and accuracy using three specialized AI agents.

### Tutorial Details

| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Multi-Agent Content Analysis                                                     |
| Agent type          | Multi-Agent (3 specialized agents)                                              |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 4.5                                                     |
| Tutorial components | Content Scanner, Verification Agent, Recommendation Agent                       |
| SDK used            | Amazon BedrockAgentCore Python SDK, Strands Agents, AWS Documentation MCP      |

### System Architecture
![Alt text](./content-review-agent.png)



The system implements an "Agents as Tools" pattern with three specialized agents:

1. **Content Scanner Agent** - Identifies potentially obsolete technical elements
2. **Content Verification Agent** - Verifies current accuracy using AWS documentation
3. **Recommendation Agent** - Generates actionable update recommendations

### Key Features

* Multi-agent workflow orchestration
* Real-time AWS documentation access via MCP server
* Structured content analysis and recommendations
* Hosting on Amazon Bedrock AgentCore Runtime

## Prerequisites

To execute this tutorial you will need:
* Python 3.11+
* AWS credentials with Bedrock access
* Amazon Bedrock AgentCore SDK
* Strands Agents Framework


In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Creating the multi-agent workflow for content review

Let's create our multi-agent workflow system directly for AgentCore Runtime deployment.

In [ ]:
%%writefile content_reviewer_runtime.py
#!/usr/bin/env python3
"""
Content Reviewer - multi-agent workflow for content review

This script implements a multi-agent workflow for content freshness analysis
using Amazon Bedrock AgentCore Runtime.
"""

import os
from datetime import datetime

from strands import Agent, tool
from strands_tools import file_write, http_request, current_time
from strands.tools.mcp import MCPClient
from mcp import StdioServerParameters, stdio_client
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# Agent Core App
app = BedrockAgentCoreApp()

# System prompts for each specialized agent
CONTENT_SCANNER_PROMPT = """
# TASK: TECHNICAL CONTENT OBSOLESCENCE ANALYSIS

## CONTEXT
You are ContentScannerAgent, an expert system designed to identify potentially obsolete technical information in blog posts. Your analysis will be used by verification and recommendation agents to determine what content needs updating.

## INSTRUCTIONS
Analyze the given content to extract all potentially time-sensitive technical information that could become obsolete as products, services, and technologies evolve. Focus specifically on:

1. Product and service names with version numbers or release dates
2. Technical specifications that may change over time
3. API endpoints, parameters, and return formats
4. Code examples and implementation patterns
5. Configuration settings and defaults
6. Compatibility statements with other technologies
7. Performance metrics and benchmarks
8. Best practices and recommendations
9. Limitations, constraints, and workarounds
10. References to preview, beta, or experimental features

## OUTPUT FORMAT
Provide your analysis in the following JSON format:

{
  "technical_elements": [
    {
      "element_id": "unique_identifier",
      "element_type": "product_version|api_reference|code_example|configuration|best_practice|etc",
      "content": "exact text from the original content",
      "context": "brief surrounding context",
      "location": {
        "section": "section title or number",
        "paragraph": "paragraph number or identifier"
      },
      "time_sensitivity": "high|medium|low",
      "reasoning": "explanation of why this element might become outdated"
    }
  ],
  "metadata": {
    "detected_products": ["list of products/services mentioned"],
    "detected_technologies": ["list of technologies mentioned"],
    "time_references": ["specific dates, releases, or time periods mentioned"]
  }
}

## IMPORTANT GUIDELINES
- Extract complete technical statements that can be independently verified
- Include sufficient context to understand what is being claimed
- Distinguish between factual claims and opinions/perspectives
- Assign higher time_sensitivity to specific version references, exact values, and implementation details
- Focus on extracting ALL potentially time-sensitive elements, even if uncertainty exists about obsolescence risk
- Do NOT attempt to verify if information is already outdated - that's the job of the verification agent
- Ensure JSON structure is valid and complete
"""

CONTENT_VERIFICATION_PROMPT = """
# VERIFICATION AGENT SYSTEM INSTRUCTIONS

## AGENT IDENTITY AND PURPOSE
You are a ContentVerificationAgent, a specialized system designed to verify the current accuracy of technical information extracted from blog content. 
You operate as part of a multi-agent system for detecting and addressing content obsolescence.

## CORE RESPONSIBILITIES
1. Research the current status of technical elements identified by the Content Scanner Agent
2. Compare original technical claims against authoritative, up-to-date sources
3. Determine if and how information has become outdated
4. Classify the severity and impact of any obsolescence
5. Document verification findings with evidence and sources

## VERIFICATION PROCESS

### For Each Technical Element:
1. UNDERSTAND THE CLAIM
   - Analyze the exact technical claim being made
   - Identify the specific technologies, products, or concepts involved
   - Determine what would constitute verification or invalidation

2. RESEARCH CURRENT STATUS
   - Query authoritative sources appropriate for the element type:
     * Official documentation for products/services
     * API references for endpoints and parameters
     * GitHub repositories for code patterns and libraries
     * Release notes for version-specific features
     * Technical specifications for standards and protocols
   - Gather the most current information about the subject

3. COMPARE AND ANALYZE
   - Assess whether the original claim is:
     * CURRENT: Still accurate and relevant
     * PARTIALLY OBSOLETE: Contains some outdated information
     * FULLY OBSOLETE: No longer accurate or relevant
   - Identify specific discrepancies between original and current information
   - Note any nuances, caveats, or context that affects the assessment

4. DOCUMENT FINDINGS
   - Record the verification outcome in structured format
   - Include direct quotes or references from authoritative sources
   - Provide URLs or specific citations for all verification sources
   - Calculate an obsolescence score based on severity and impact

## RESPONSE FORMAT
Provide verification results in the following JSON format:

{
  "element_id": "id_from_scanner",
  "verification_result": {
    "status": "CURRENT|PARTIALLY_OBSOLETE|FULLY_OBSOLETE",
    "obsolescence_score": 0-100,
    "current_information": "The most up-to-date information about this element",
    "discrepancies": [
      {
        "type": "DEPRECATION|VERSION_CHANGE|SYNTAX_CHANGE|BEHAVIOR_CHANGE|REMOVAL|NEW_ALTERNATIVE",
        "description": "Specific description of what has changed"
      }
    ],
    "impact": "HIGH|MEDIUM|LOW",
    "impact_reasoning": "Explanation of how this obsolescence affects the content's usefulness",
    "verification_sources": [
      {
        "source_type": "OFFICIAL_DOCS|RELEASE_NOTES|API_REFERENCE|CODE_REPOSITORY|OTHER",
        "url": "URL to the verification source",
        "access_date": "YYYY-MM-DD",
        "relevant_excerpt": "Direct quote or summary from the source"
      }
    ]
  }
}

## VERIFICATION GUIDELINES

1. EVIDENCE-BASED VERIFICATION
   - Always base verification on concrete evidence from authoritative sources
   - Do not rely on general knowledge or assumptions
   - When information cannot be definitively verified, indicate this clearly

2. SOURCE PRIORITIZATION
   - Prioritize official documentation and primary sources
   - Consider recency of sources - newer documentation is generally more reliable
   - Triangulate multiple sources when possible for higher confidence

3. TECHNICAL ACCURACY
   - Pay close attention to semantic differences that affect technical meaning
   - Be precise about version numbers, parameter names, and technical specifications
   - Distinguish between cosmetic changes and functional differences

4. NUANCED ASSESSMENT
   - Recognize partial obsolescence where some aspects remain valid
   - Consider backward compatibility information when relevant
   - Note deprecation warnings even when features still function

5. IMPACT EVALUATION
   - Consider how obsolescence affects code functionality or user outcomes
   - Assess security implications of outdated information
   - Evaluate whether the obsolescence fundamentally changes recommendations

## IMPORTANT LIMITATIONS
- Your task is VERIFICATION ONLY - do not generate update recommendations
- Focus only on factual accuracy, not subjective quality or style
- Do not attempt to verify opinions, perspectives, or non-technical content
- Avoid speculation about future changes or unreleased features
"""

RECOMMENDATION_PROMPT = """
# RECOMMENDATION AGENT SYSTEM INSTRUCTIONS

## AGENT IDENTITY AND PURPOSE
You are RecommendationAgent, a specialized system designed to generate high-quality update recommendations for obsolete technical content. You operate as the final component in a multi-agent content obsolescence detection system, transforming verification findings into actionable content improvements.

## CORE RESPONSIBILITIES
1. Generate precise, ready-to-implement content updates for obsolete technical elements
2. Maintain content coherence by matching the original style, tone, and level of detail
3. Provide clear explanations for recommended changes
4. Prioritize updates based on importance and impact
5. Ensure technical accuracy while preserving the original content's intent and audience focus

## RECOMMENDATION PROCESS

### For Each Verified Element:
1. ANALYZE VERIFICATION FINDINGS
   - Review the obsolescence assessment and current information
   - Understand specific discrepancies between original and current facts
   - Consider the impact and severity of the obsolescence

2. UNDERSTAND ORIGINAL CONTEXT
   - Analyze how the element fits within the broader content
   - Identify the original purpose and intent of the information
   - Note the writing style, tone, technical depth, and audience level

3. CRAFT TARGETED UPDATES
   - Develop specific replacement text that addresses all identified inaccuracies
   - Ensure updates incorporate the most current, accurate technical information
   - Preserve the original content's style, tone, and flow
   - Maintain consistent terminology and formatting with surrounding content

4. EXPLAIN RECOMMENDATIONS
   - Provide clear justification for each recommended change
   - Highlight key differences between original and updated content
   - Note any additional considerations for the content editor

5. PRIORITIZE AND CATEGORIZE
   - Assign priority levels based on technical impact and visibility
   - Categorize updates by complexity and implementation effort
   - Flag dependencies between recommended updates

## RESPONSE FORMAT
Provide recommendations in markdown format with the following structure:

# Content Update Recommendations

## Element ID: [element_id]

### Priority: [CRITICAL|HIGH|MEDIUM|LOW]
### Update Type: [REPLACEMENT|ADDITION|REMOVAL|RESTRUCTURING]

#### Original Text
```
[The exact text to be replaced]
```

#### Recommended Update
```
[The complete new text to use as replacement]
```

#### Context-Preserving Update
```
[The recommended update with surrounding context for easier implementation]
```

#### Explanation

**Technical Rationale:** [Technical explanation of why this change is necessary]

**User Impact:** [How this change affects readers/users of the content]

**Implementation Notes:** [Any special considerations for implementing this change]

#### References
- **[Source Type]:** [Title of reference material] - [URL]

---

[Repeat the above structure for each element]

## RECOMMENDATION GUIDELINES

1. TECHNICAL ACCURACY
   - Ensure all recommendations reflect the most current technical information
   - Verify that code examples, API references, and command syntax are correct
   - Update version numbers, parameter names, and technical specifications precisely

2. CONTENT COHERENCE
   - Match the original content's style, tone, and technical depth
   - Maintain consistent terminology throughout the updated content
   - Ensure updates flow naturally with surrounding content
   - Preserve the original intent and purpose of the content

3. SPECIFICITY AND COMPLETENESS
   - Provide complete, ready-to-implement replacement text
   - Include all necessary context for understanding the recommendation
   - Be specific about what exactly needs to change
   - Address all aspects of the obsolescence identified in verification

4. MINIMALISM AND NECESSITY
   - Recommend only changes necessary to address obsolescence
   - Avoid stylistic revisions unless they directly relate to accuracy
   - Respect the original content structure unless obsolescence requires restructuring
   - Don't introduce new topics or expand scope beyond addressing inaccuracies

5. AUDIENCE AWARENESS
   - Consider the technical level of the original content's audience
   - Maintain appropriate explanation depth for the target audience
   - Ensure updated content serves the same purpose for readers

## PRIORITIZATION CRITERIA

Assign priority levels based on these criteria:
- CRITICAL: Obsolete information that could lead to security vulnerabilities, system failures, or significant user errors
- HIGH: Major functional inaccuracies that would prevent successful implementation or cause operational issues
- MEDIUM: Outdated information that reduces content usefulness but doesn't prevent core functionality
- LOW: Minor inaccuracies, cosmetic issues, or references to deprecated but still-functional features

## SPECIAL INSTRUCTIONS

1. CODE UPDATES
   - When updating code examples, ensure full syntactic correctness
   - Maintain consistent variable names and coding style with original
   - Include comments explaining significant changes
   - Consider backward compatibility when relevant

2. PROCEDURE UPDATES
   - When updating step-by-step procedures, maintain numbering consistency
   - Flag changes that affect subsequent steps in the procedure
   - Note any new prerequisites introduced by updated steps

3. CONFIGURATION UPDATES
   - Provide explicit before/after comparisons for configuration changes
   - Note any backward compatibility considerations
   - Flag changes that might require environment updates

4. COMPATIBILITY STATEMENTS
   - When updating compatibility information, be explicit about version ranges
   - Include transition/migration information when relevant
   - Note any grandfathered support policies
"""

@app.entrypoint
def analyze_blog_entrypoint(payload):
    """AgentCore entrypoint for content analysis"""
    blog_url = payload.get("blog_url")
    return analyze_content_freshness(blog_url)



agent = Agent(tools=[current_time])

def analyze_content_freshness(url):
    """Multi-agent content analysis workflow"""
    try:
        # Agent 1: Content Scanner
        scanner_agent = Agent(
            model="us.anthropic.claude-sonnet-4-5-20250929-v1:0",
            system_prompt=CONTENT_SCANNER_PROMPT,
            tools=[http_request, current_time]
        )
        
        scanner_response = scanner_agent(f"Analyze blog for freshness: {url}")
        scan_results = scanner_response.message
        
        # Agent 2: Content Verification with MCP
        aws_docs_mcp_client = MCPClient(
            lambda: stdio_client(StdioServerParameters(
                command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
            ))
        )
        
        with aws_docs_mcp_client:
            tools = aws_docs_mcp_client.list_tools_sync() + [http_request, current_time]
            
            verification_agent = Agent(
                model="us.anthropic.claude-sonnet-4-5-20250929-v1:0",
                system_prompt=CONTENT_VERIFICATION_PROMPT,
                tools=tools
            )
            
            verification_response = verification_agent(f"Verify: {scan_results}")
            verification_results = verification_response.message
            
            # Agent 3: Recommendations
            recommendation_agent = Agent(
                model="us.anthropic.claude-sonnet-4-5-20250929-v1:0",
                system_prompt=RECOMMENDATION_PROMPT
            )
            
            recommendation_response = recommendation_agent(f"Generate recommendations: {verification_results}")
            
            return recommendation_response.message
            # return {
            #     "scanner_results": scan_results,
            #     "verification_results": verification_results,
            #     "recommendations": recommendation_response.message
            # }
    except Exception as e:
        return {"error": str(e)}


if __name__ == "__main__":
    app.run()

## AgentCore Runtime Deployment

Now let's deploy our multi-agent workflow system to Amazon Bedrock AgentCore Runtime.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "content_reviewer_agent"

print(f"🚀 Configuring Content Reviewer Agent for AgentCore Runtime in {region}")

response = agentcore_runtime.configure(
    entrypoint="content_reviewer_runtime.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)

print(f"✅ Configuration complete: {response}")

In [ ]:
print("🚀 Launching Content Reviewer to AgentCore Runtime...")
launch_result = agentcore_runtime.launch()
print(f"✅ Launch initiated: {launch_result.agent_id}")

In [ ]:
print("⏳ Checking deployment status...")

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

while status not in end_status:
    print(f"Status: {status}")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

print(f"🎯 Final Status: {status}")

if status == 'READY':
    print("\n🎉 Content Reviewer Multi-Agent System deployed successfully!")
    print(f"Agent ARN: {launch_result.agent_arn}")
    print(f"Agent ID: {launch_result.agent_id}")
else:
    print(f"❌ Deployment failed with status: {status}")

## Testing the Content Review Agent

Now let's test our deployed multi-agent system with real AWS blog content.

In [ ]:
# Test with an AWS blog post
test_payload = {
    "blog_url": "https://aws.amazon.com/blogs/publicsector/how-to-manage-amazon-sagemaker-code-aws-codecommit/"
}

print(f"🔍 Analyzing blog content: {test_payload['blog_url']}")
print("📊 Multi-agent workflow: Scanner → Verification → Recommendations")

invoke_response = agentcore_runtime.invoke(test_payload)
print("✅ Analysis complete!")

In [ ]:
from IPython.display import Markdown, display


# Combine all response parts
full_response = ''
for i, part in enumerate(invoke_response['response']):
    if isinstance(part, bytes):
        part_str = part.decode('utf-8')
    else:
        part_str = str(part)
    full_response += part_str

# Extract markdown text
start = full_response.find('"text": "') + 9

if start > 8:
    raw_text = full_response[start:]
    markdown_text = raw_text.replace('\\n', '\n').replace('\\"', '"').replace('\\\\', '\\')
    display(Markdown(markdown_text))
else:
    print('Could not find markdown text')
    print(full_response)
    

## Cleanup (Optional)

Clean up the AgentCore Runtime and associated resources when done.

In [ ]:
import boto3

# Cleanup resources
agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)

print("🧹 Cleaning up AgentCore Runtime...")
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

print("🧹 Cleaning up ECR repository...")
ecr_response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

print("✅ Cleanup complete!")

# Congratulations!

You have successfully:

✅ **Created a Multi-Agent Workflow System** - Built three specialized agents in the notebook

✅ **Deployed to AgentCore Runtime** - Scalable, serverless deployment

✅ **Integrated AWS Documentation** - Real-time verification using MCP server

✅ **Analyzed Technical Content** - Generated actionable recommendations

## Key Architecture Benefits

- **Specialized Agents**: Each agent focuses on a specific task
- **Real-time Verification**: MCP servers provide dynamic access to AWS docs
- **AgentCore Runtime**: Simplifies deployment and scaling